## PATH SETUP

In [1]:
# MUST be first cell: set multiprocessing method for Windows
import torch.multiprocessing as mp
try:
    mp.set_start_method("spawn", force=True)
except RuntimeError:
    pass  # Already set

In [2]:
import os
import sys
from pathlib import Path

# FOR LOCAL USE THIS LINES
current = Path.cwd()
src_path = current / "src" if (current / "src").exists() else current.parent

# FOR COLAB USE THIS LINE INSTEAD
# BRANCH_NAME = "main"  # Change this to switch branches
# !git clone -b {BRANCH_NAME} https://github.com/MatteoCamillo-code/GeoLoc-CVCS.git
# !cd /content/GeoLoc-CVCS && git pull origin {BRANCH_NAME} && cd ..
# src_path = Path("/content/GeoLoc-CVCS/src").resolve()

sys.path.insert(0, str(src_path))

from utils.paths import find_project_root

# Set working directory and sys.path properly
project_root = find_project_root(src_path)
data_dir = project_root / "data"
history_dir = project_root / "outputs" / "history"
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
print("CWD:", Path.cwd())

CWD: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS


## IMPORT

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

from configs.baseline_multi_head_ISN import TrainConfig

cfg = TrainConfig()
if cfg.backbone == "resnet50":
    from torchvision.models import resnet50, ResNet50_Weights
elif cfg.backbone == "inceptionv4":
    import timm

from utils.seed import seed_everything
from utils.metrics import overall_val_acc_from_history
from utils.io import save_json, read_json
from utils.paths import get_next_version, get_current_version
from training.runner import fit
from training.losses import CrossEntropyWithLabelSmoothing

from src.utils.logging import get_logger
from src.utils.paths import abs_path

from models.multi_head_classifier import MultiHeadClassifier


In [ ]:
seed_everything(cfg.seed)

device = cfg.device if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [5]:
import kagglehub

path = kagglehub.dataset_download("josht000/osv-mini-129k")
path = path + "/osv5m"
print("Path to dataset files:", path)

image_root = path + "/train_images"


c:\Users\camil\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\camil\.cache\kagglehub\datasets\josht000\osv-mini-129k\versions\1/osv5m


In [6]:
train_val_path = data_dir / "metadata/s2-geo-cells/train_val_split_geocells.csv"
cell_centers_path = data_dir / "metadata/s2-geo-cells/cell_center_dataset.csv"

train_val_meta = pd.read_csv(train_val_path)
cell_centers_df = pd.read_csv(cell_centers_path)

# remove duplicates with same cell_id
cell_centers_df = cell_centers_df.drop_duplicates(subset='cell_id_token')

# Set S2 cell ID as index (assumes first column or 'cell_id' column)
if 'cell_id_token' in cell_centers_df.columns:
    cell_centers_df = cell_centers_df.set_index('cell_id_token')
else:
    # Set first column as index if it contains cell IDs
    cell_centers_df = cell_centers_df.set_index(cell_centers_df.columns[0])
    

print("Train/val CSV:", train_val_path)
print("Cell centers CSV:", cell_centers_path)


Train/val CSV: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\data\metadata\s2-geo-cells\train_val_split_geocells.csv
Cell centers CSV: F:\InfTech\Prodotti\Python\GeoLocGit\GeoLoc-CVCS\data\metadata\s2-geo-cells\cell_center_dataset.csv


## DATALOADER

In [7]:
from dataset.dataloader_utils import create_dataloaders

IMG_SIZE = 224

# Create all dataloaders with a single function call
loader_dict = create_dataloaders(
    image_root=image_root,
    csv_path=train_val_path,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    img_size=IMG_SIZE,
    seed=cfg.seed,
    train_subset_pct=cfg.train_size_pct,
    val_subset_pct=cfg.val_size_pct,
    scenes=cfg.scenes,
    augment=True,
    prefetch_factor=4,
    persistent_workers=True if cfg.num_workers > 0 else False,
    coarse_label_idx=cfg.coarse_label_idx,
)

## MODEL

In [ ]:
models = {}

for sc in cfg.scenes:
    backbone = None
    FEAT_DIM = 0
    if cfg.backbone == "inceptionv4":
        backbone = timm.create_model('inception_v4', pretrained=True)
        FEAT_DIM = 1536  # inception_v4 feature dimension
    elif cfg.backbone == "resnet50":
        weights = ResNet50_Weights.IMAGENET1K_V2
        backbone = resnet50(weights=weights)
        FEAT_DIM = 2048  # resnet50 feature dimension  

    # number of classes depends on partition
    num_classes = list(map(
        lambda idx: len(loader_dict[sc]["label_maps"][f"label_config_{idx + 1}"]),
        cfg.coarse_label_idx
    ))

    backbone = nn.Sequential(
        *list(backbone.children())[:-1],
        nn.Flatten(1)
    )


    backbone = backbone.to(device)
    # Optional: comment out if it causes issues on Windows/your PyTorch version
    # model = torch.compile(model, backend="aot_eager")

    model = MultiHeadClassifier(
        backbone=backbone,
        feat_dim=FEAT_DIM,
        head_dims=num_classes,
        dropout=cfg.dropout,
        coarse_level_idx=cfg.coarse_label_idx,
    ).to(device)
    
    models[sc] = model

    print(f"Output classes {sc}:", num_classes)


Output classes urban: [3101, 1850, 1091]
Output classes natural: [4561, 2472, 1328]


In [9]:
criterion = CrossEntropyWithLabelSmoothing(ignore_index=-1, smoothing=cfg.label_smoothing)

# Create optimizer, scheduler, and scaler for each model
optimizers = {}
schedulers = {}
scalers = {}

for scene, model in models.items():
    optimizers[scene] = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    schedulers[scene] = StepLR(optimizers[scene], step_size=cfg.scheduler_step_size, gamma=cfg.scheduler_gamma)
    scalers[scene] = torch.amp.GradScaler(device=cfg.device, enabled=cfg.amp)

torch.backends.cudnn.benchmark = True
print(f"Initialized training components for {len(models)} scenes: {list(models.keys())}")

Initialized training components for 2 scenes: ['urban', 'natural']


## TRAINING LOOP

In [10]:
# Train each scene's model separately
histories = {}
logger = get_logger(log_file=str(abs_path(cfg.output_dir, "logs", "train.log")))
version = 0

for scene in models.keys():
    # Get versioned history path
    base_name = f"{cfg.model_name}_{scene}"
    version = get_next_version(history_dir, base_name)
    history_path = history_dir / f"{base_name}_v{version}_history.json"
    
    history = fit(
        cfg=cfg,
        model=models[scene],
        data_loader=loader_dict[scene],
        cell_centers=cell_centers_df,
        optimizer=optimizers[scene],
        criterion=criterion,
        scaler=scalers[scene],
        use_tqdm=cfg.use_tqdm,
        scheduler=schedulers[scene],
        logger=logger,
        scene=scene,
        history_path=history_path,
        version=version,
    )
    
    histories[scene] = history

[15:50:23] INFO - Starting training baseline_multi_head_ISN for scene urban ...
[15:51:14] INFO - Epoch 1/3 | train loss=7.3188 acc=0.95% | val loss=7.1564 acc=2.14% | geo acc={'acc@1km': '0.51%', 'acc@5km': '4.83%', 'acc@25km': '20.87%', 'acc@100km': '26.46%'} | time=50.4s
[15:51:33] INFO - Epoch 2/3 | train loss=6.7475 acc=8.15% | val loss=7.0056 acc=4.35% | geo acc={'acc@1km': '1.27%', 'acc@5km': '9.10%', 'acc@25km': '25.25%', 'acc@100km': '32.25%'} | time=19.4s
[15:51:53] INFO - Epoch 3/3 | train loss=6.3808 acc=19.40% | val loss=6.8949 acc=5.45% | geo acc={'acc@1km': '2.16%', 'acc@5km': '11.51%', 'acc@25km': '31.17%', 'acc@100km': '37.72%'} | time=19.6s
[15:51:53] INFO - Training completed.
[15:51:53] INFO - Starting training baseline_multi_head_ISN for scene natural ...
[15:53:04] INFO - Epoch 1/3 | train loss=7.4426 acc=5.00% | val loss=7.0849 acc=11.15% | geo acc={'acc@1km': '0.21%', 'acc@5km': '5.55%', 'acc@25km': '16.06%', 'acc@100km': '27.73%'} | time=70.4s
[15:53:51] INFO -

In [11]:
# version = 3
accuracy_list = overall_val_acc_from_history(cfg, project_root, version)
print(accuracy_list)

{'urban': 0.05449533462524414, 'natural': 0.22947627305984497, 'overall': 0.17796503050943438}
